#  Model Training – OpenAI Embeddings + Pinecone

Pipeline:
1) Load env keys from `Backend/.env`
2) Load dataset
3) Build embedding text (title | description | brand | categories)
4) Create/connect Pinecone index (1536 dims)
5) Upsert vectors + metadata
6) Test a query


In [10]:
# ✅ Notebook-safe setup for OpenAI + Pinecone
import os
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI

# --------- PATHS (no __file__) ----------
# Try current dir; if no Backend here, try parent (common layout: Frontend/notebook.ipynb)
cwd = Path.cwd()
if (cwd / "Backend").exists():
    BASE_DIR = cwd
elif (cwd.parent / "Backend").exists():
    BASE_DIR = cwd.parent
else:
    # Fallback: customize if your tree is different
    BASE_DIR = cwd  # still allow load_dotenv() to search upward
    print("⚠️ Couldn't find 'Backend' next to notebook. Using current directory; "
          "adjust BASE_DIR if needed.")

ENV_PATH = BASE_DIR / "Backend" / ".env"
CSV_PATH = BASE_DIR / "Backend" / "data" / "intern_data_ikarus.csv"

# --------- CONFIG ----------
EMBED_MODEL = "text-embedding-3-small"  # 1536 dims
DESIRED_DIM = 1536

# --------- ENV LOAD ----------
# If explicit .env not found, load_dotenv() will still search upward from CWD
if ENV_PATH.exists():
    ok = load_dotenv(ENV_PATH)
else:
    print(f"⚠️ .env not found at {ENV_PATH}. Falling back to load_dotenv() search.")
    ok = load_dotenv()

assert ok, f"Could not load .env (looked for {ENV_PATH} and upward search)."

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX  = os.getenv("PINECONE_INDEX", "ml-project1")
PINECONE_CLOUD  = os.getenv("PINECONE_CLOUD", "aws")
PINECONE_REGION = os.getenv("PINECONE_REGION", "us-east-1")

assert OPENAI_API_KEY, "OPENAI_API_KEY missing in Backend/.env (or environment)"
assert PINECONE_API_KEY, "PINECONE_API_KEY missing in Backend/.env (or environment)"

# Optional sanity checks
print("ENV PATH exists:", ENV_PATH.exists())
print("CSV PATH exists:", CSV_PATH.exists(), "->", CSV_PATH)

# --------- CLIENTS ----------
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
client = OpenAI()

pc = Pinecone(api_key=PINECONE_API_KEY)

# --------- INDEX CREATE/GET (handles SDK variations) ----------
def list_index_names(pc_client):
    """
    Pinecone SDK has had a few return shapes:
    - some versions: list of dicts with 'name'
    - others: objects with .name
    - newer: pc.list_indexes().names()
    """
    try:
        # Newer helper if available
        return set(pc_client.list_indexes().names())
    except Exception:
        items = pc_client.list_indexes()
        names = set()
        for i in items:
            if isinstance(i, dict) and "name" in i:
                names.add(i["name"])
            elif hasattr(i, "name"):
                names.add(i.name)
        return names

existing = list_index_names(pc)

if PINECONE_INDEX not in existing:
    print(f"Creating index '{PINECONE_INDEX}' ({DESIRED_DIM} dims, cosine)...")
    pc.create_index(
        name=PINECONE_INDEX,
        dimension=DESIRED_DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud=PINECONE_CLOUD, region=PINECONE_REGION),
    )
else:
    # describe_index() can return different shapes too; guard it
    info = pc.describe_index(PINECONE_INDEX)
    # Try common access patterns
    dim = getattr(info, "dimension", None)
    if dim is None:
        dim = info.get("dimension") if isinstance(info, dict) else None
    if dim is None:
        # Last resort: skip strict check but warn
        print("⚠️ Couldn't read index dimension from describe_index(); skipping dim check.")
    elif dim != DESIRED_DIM:
        raise RuntimeError(
            f"Index '{PINECONE_INDEX}' has dim {dim}, expected {DESIRED_DIM}. "
            f"Recreate the index or change EMBED_MODEL."
        )

index = pc.Index(PINECONE_INDEX)
print("✅ Pinecone index ready:", PINECONE_INDEX)


ENV PATH exists: True
CSV PATH exists: True -> d:\ml-product-app\Backend\data\intern_data_ikarus.csv
✅ Pinecone index ready: ml-project1


In [11]:
print("Loading:", os.path.abspath(CSV_PATH))
df = pd.read_csv(CSV_PATH)
print("Rows:", len(df))
df.head()


Loading: d:\ml-product-app\Backend\data\intern_data_ikarus.csv
Rows: 312


title            brand  \
0  GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...           GOYMFK   
1  subrtex Leather ding Room, Dining Chairs Set o...          subrtex   
2  Plant Repotting Mat MUYETOL Waterproof Transpl...          MUYETOL   
3  Pickleball Doormat, Welcome Doormat Absorbent ...          VEWETOL   
4  JOIN IRON Foldable TV Trays for Eating Set of ...  JOIN IRON Store   

                                         description   price  \
0  multiple shoes, coats, hats, and other items E...  $24.99   
1                     subrtex Dining chairs Set of 2     NaN   
2                                                NaN   $5.98   
3  The decorative doormat features a subtle textu...  $13.99   
4  Set of Four Folding Trays With Matching Storag...  $89.99   

                                          categories  \
0  ['Home & Kitchen', 'Storage & Organization', '...   
1  ['Home & Kitchen', 'Furniture', 'Dining Room F...   
2  ['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...   
3  ['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...   
4  ['Home & Kitchen', 'Furniture', 'Game & Recrea...   

                                              images           manufacturer  \
0  ['https://m.media-amazon.com/images/I/416WaLx1...                 GOYMFK   
1  ['https://m.media-amazon.com/images/I/31SejUEW...  Subrtex Houseware INC   
2  ['https://m.media-amazon.com/images/I/41RgefVq...                MUYETOL   
3  ['https://m.media-amazon.com/images/I/61vz1Igl...              Contrence   
4  ['https://m.media-amazon.com/images/I/41p4d4VJ...                    NaN   

         package_dimensions country_of_origin      material          color  \
0  2.36"D x 7.87"W x 21.6"H             China         Metal          White   
1      18.5"D x 16"W x 35"H               NaN        Sponge          Black   
2           26.8"L x 26.8"W               NaN  Polyethylene          Green   
3               24"L x 16"W               NaN        Rubber          A5589   
4    18.9"D x 14.2"W x 26"H               NaN          Iron  Grey Set of 4   

                                uniq_id  
0  02593e81-5c09-5069-8516-b0b29f439ded  
1  5938d217-b8c5-5d3e-b1cf-e28e340f292e  
2  b2ede786-3f51-5a45-9a5b-bcf856958cd8  
3  8fd9377b-cfa6-5f10-835c-6b8eca2816b5  
4  bdc9aa30-9439-50dc-8e89-213ea211d66a

In [13]:
# price numeric
if "price" in df.columns:
    df["price_num"] = pd.to_numeric(
        df["price"].astype(str).str.replace("$", "", regex=False).str.replace(",", "", regex=False).str.strip(),
        errors="coerce"
    )
else:
    df["price_num"] = None

# ensure unique id
id_col = "uniq_id" if "uniq_id" in df.columns else None
if id_col is None:
    df["uniq_id"] = df.index.astype(str)
    id_col = "uniq_id"

# build text for embedding
import pandas as pd
import math

def is_nan(x):
    try:
        return x is None or pd.isna(x)
    except Exception:
        return False

def safe_str(x, max_len=2000):
    """Return '' for None/NaN, else trimmed string."""
    if is_nan(x):
        return ""
    return str(x)[:max_len]

# MAIN: build the text we send to OpenAI embeddings
def build_text(row):
    parts = [
        safe_str(row.get("title")),
        safe_str(row.get("description")),
    ]
    if not is_nan(row.get("brand")):
        parts.append(safe_str(row.get("brand")))
    if not is_nan(row.get("categories")):
        parts.append(safe_str(row.get("categories")))
    return " | ".join([p for p in parts if p])

# Recompute embed_text safely
df["embed_text"] = df.apply(build_text, axis=1)
df[["uniq_id", "title", "brand", "categories", "price", "price_num", "embed_text"]].head()




uniq_id  \
0  02593e81-5c09-5069-8516-b0b29f439ded   
1  5938d217-b8c5-5d3e-b1cf-e28e340f292e   
2  b2ede786-3f51-5a45-9a5b-bcf856958cd8   
3  8fd9377b-cfa6-5f10-835c-6b8eca2816b5   
4  bdc9aa30-9439-50dc-8e89-213ea211d66a   

                                               title            brand  \
0  GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...           GOYMFK   
1  subrtex Leather ding Room, Dining Chairs Set o...          subrtex   
2  Plant Repotting Mat MUYETOL Waterproof Transpl...          MUYETOL   
3  Pickleball Doormat, Welcome Doormat Absorbent ...          VEWETOL   
4  JOIN IRON Foldable TV Trays for Eating Set of ...  JOIN IRON Store   

                                          categories   price  price_num  \
0  ['Home & Kitchen', 'Storage & Organization', '...  $24.99      24.99   
1  ['Home & Kitchen', 'Furniture', 'Dining Room F...     NaN        NaN   
2  ['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...   $5.98       5.98   
3  ['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...  $13.99      13.99   
4  ['Home & Kitchen', 'Furniture', 'Game & Recrea...  $89.99      89.99   

                                          embed_text  
0  GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...  
1  subrtex Leather ding Room, Dining Chairs Set o...  
2  Plant Repotting Mat MUYETOL Waterproof Transpl...  
3  Pickleball Doormat, Welcome Doormat Absorbent ...  
4  JOIN IRON Foldable TV Trays for Eating Set of ...

In [6]:
def embed_text(text: str, model: str = EMBED_MODEL):
    text = (text or "").replace("\n", " ")
    resp = client.embeddings.create(model=model, input=[text])
    return resp.data[0].embedding


In [14]:
def safe_num(x):
    """Return float or None (never NaN)."""
    return None if is_nan(x) else float(x)

def safe_meta(row):
    """Return metadata dict with no NaN/invalid JSON values."""
    meta = {
        "title": safe_str(row.get("title")),
        "brand": (None if is_nan(row.get("brand")) else safe_str(row.get("brand"))),
        "category": (None if is_nan(row.get("categories")) else safe_str(row.get("categories"))),
        "description": safe_str(row.get("description")),
        "price": safe_num(row.get("price_num")),
    }
    if "images" in row and not is_nan(row.get("images")):
        meta["image_url"] = safe_str(row.get("images"))
    return meta


In [15]:
from tqdm import tqdm

BATCH = 100
batch = []
skipped = 0

print("Upserting vectors...")
for _, row in tqdm(df.iterrows(), total=len(df)):
    try:
        vec = embed_text(row["embed_text"])      # your existing function
        meta = safe_meta(row)                     # <- JSON-safe metadata
        vec_obj = {
            "id": str(row[id_col]),              # make sure id_col is set earlier
            "values": vec,
            "metadata": meta
        }
        batch.append(vec_obj)
    except Exception as e:
        skipped += 1
        # print("Embed failed for:", row[id_col], e)
        continue

    if len(batch) >= BATCH:
        try:
            index.upsert(vectors=batch)
        except Exception:
            # fall back to one-by-one to skip any bad item
            for v in batch:
                try:
                    index.upsert(vectors=[v])
                except Exception:
                    skipped += 1
        batch = []

# flush remainder
if batch:
    try:
        index.upsert(vectors=batch)
    except Exception:
        for v in batch:
            try:
                index.upsert(vectors=[v])
            except Exception:
                skipped += 1

print("Upsert complete. Skipped rows (embedding or metadata errors):", skipped)


Upserting vectors...


100%|██████████| 312/312 [04:38<00:00,  1.12it/s]


Upsert complete. Skipped rows (embedding or metadata errors): 97


In [17]:
# Helper function: Query Pinecone for similar products
def query_pinecone(text, top_k=5):
    # Create embedding for the user query
    qvec = embed_text(text)
    # Search Pinecone
    res = index.query(vector=qvec, top_k=top_k, include_metadata=True)
    return res


res = query_pinecone("wooden chair", top_k=5)

rows = []
for m in res.get("matches", []):
    md = m.get("metadata", {}) or {}
    rows.append({
        "id": m.get("id"),
        "score": m.get("score"),
        "title": md.get("title"),
        "brand": md.get("brand"),
        "price": md.get("price"),
        "category": md.get("category"),
        "desc_snippet": (md.get("description") or "")[:150]
    })

import pandas as pd
pd.DataFrame(rows)


id     score  \
0  8b85369f-c20b-5e61-baf3-46ead5b2dba7  0.469584   
1  f017c252-c086-5b08-8b44-ec475a2cb217  0.467290   
2  0acd8c1c-d689-5fc2-b0ba-9909a60f47dd  0.440414   
3  d5a75f7b-b874-5757-9778-790a1f33be14  0.440125   
4  fe25ae1d-4a82-57ad-9bab-b9de4321fd0b  0.439235   

                                               title            brand   price  \
0  Tiita Saucer Chair with Ottoman, Soft Faux Fur...      Tiita Store  139.99   
1  BOOSDEN Padded Folding Chair 2 Pack, Foldable ...    BOOSDEN Store  119.00   
2  Lazy Chair with Ottoman, Modern Lounge Accent ...      WARMGIFT WM  139.99   
3  Tiita Comfy Saucer Chair, Soft Faux Fur Oversi...      Tiita Store   79.99   
4  Karl home Accent Chair Mid-Century Modern Chai...  Karl home Store  149.99   

                                            category  \
0  ['Home & Kitchen', 'Furniture', 'Living Room F...   
1  ['Home & Kitchen', 'Furniture', 'Game & Recrea...   
2  ['Home & Kitchen', 'Furniture', 'Living Room F...   
3  ['Home & Kitchen', 'Furniture', 'Living Room F...   
4  ['Home & Kitchen', 'Furniture', 'Living Room F...   

                                        desc_snippet  
0                                                     
1                                                     
2  Lazy Chair with Ottoman and Pillow and Blanket...  
3                                                     
4  Karl home Accent Chair Mid-Century Modern Chai...